In [ ]:
pip install transformers datasets peft trl bitsandbytes accelerate ipywidgets tensorboard optuna

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install unsloth

Note: you may need to restart the kernel to use updated packages.


In [1]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 8192,
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    load_in_16bit = True,
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
Switching to PyTorch attention since your Xformers is broken.

/home/cosmin/anaconda3/envs/rerank/lib/python3.12/site-packages/flash_attn_2_cuda.cpython-312-x86_64-linux-gnu.so: undefined symbol: _ZNK3c106SymInt6sym_neERKS0_
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA H200 NVL. Num GPUs = 1. Max memory: 139.801 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = Fals

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 128,           # Larger = higher accuracy, but might overfit
    lora_alpha = 256,  # Recommended alpha == r at least
    lora_dropout = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias = "none",
    random_state = 3407,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [4]:
from datasets import load_from_disk

dataset = load_from_disk("reranking_dataset_doris_mae")

In [5]:
dataset[100]

{'query_text': 'The document should discuss the improvement of interpretability and explainability of machine learning models.',
 'aspect_id': '171',
 'original_query_type': 'ML',
 'candidate_pool': [184834,
  141826,
  94225,
  91667,
  48153,
  84506,
  70171,
  316961,
  357922,
  193061,
  357415,
  48168,
  121899,
  64560,
  96305,
  236595,
  43574,
  152119,
  263743,
  45119,
  190015,
  242239,
  170071,
  167008,
  83044,
  173673,
  166518,
  36476,
  154247,
  261768,
  202383,
  80015,
  38037,
  80538,
  281249,
  138404,
  151208,
  208041,
  253622,
  216765,
  71871,
  44223,
  60102,
  212169,
  251597,
  271055,
  319696,
  156881,
  337109,
  160471,
  89316,
  302822,
  179957,
  207094,
  76023,
  76021,
  298758,
  162060,
  241933,
  160526,
  3852,
  99608,
  138523,
  40234,
  244011,
  69932,
  242478,
  100660,
  170805,
  272183,
  147768,
  123706,
  275272,
  67913,
  277856,
  213344,
  48992,
  166762,
  104811,
  124270,
  111989,
  91004,
  198022,
 

In [6]:
import pandas as pd

corpus_df = pd.read_json("splits/Corpus.json")

In [7]:
corpus_df.head()

,masked_abstract,original_abstract,title,url,primary_category,categories,incoming_citations,ss_id,outgoing_citations,abstract_id
0,Neural machine translation ( * ) systems aim t...,Neural machine translation (NMT) systems aim t...,How_do_lexical_semantics_affect_translation?_A...,http://arxiv.org/abs/2201.00075v1,cs.CL,"[cs.CL, cs.LG]",[],8cff7cb7a44672d1108d63ed611e1056acf2ace3,"[157157, 197007, 238926, 280486, 283908, 29077...",0
1,If popular online platforms systematically exp...,If popular online platforms systematically exp...,Engagement_Outweighs_Exposure_to_Partisan_and_...,http://arxiv.org/abs/2201.00074v3,cs.SI,[cs.SI],[],404f412bc6b76f58418604eb5caac077bb25cdac,"[14715, 57120, 84703, 111772, 124625, 263661]",1
2,While neural networks have shown remarkable su...,While neural networks have shown remarkable su...,BARACK:_Partially_Supervised_Group_Robustness_...,http://arxiv.org/abs/2201.00072v2,cs.LG,[cs.LG],[],49750bf1dd5e66025c18adfce5ce7fef445fb9d4,"[5883, 34547, 84663, 89288, 91962, 94275, 9482...",2
3,"In recent times , a large number of people hav...","In recent times, a large number of people have...",A_Deep_Learning_Approach_to_Integrate_Human-Le...,http://arxiv.org/abs/2201.02735v1,cs.CL,"[cs.CL, cs.LG]",[],0be842930627213049f5567fe37d65237e535960,"[137492, 159867, 299491]",3
4,The distributed consensus mechanism is the bac...,The distributed consensus mechanism is the bac...,Confronting_the_Carbon-footprint_Challenge_of_...,http://arxiv.org/abs/2201.06929v1,cs.CY,[cs.CY],[],1dee953d5211578d330a1328fb0033dbe644685b,"[179697, 186061, 232933, 252408]",4


In [8]:
len(corpus_df)

363133

In [9]:
print("Creating abstract_id -> text lookup dictionary...")
abstract_lookup = {}
for _, row in corpus_df.iterrows():
    title = row['title'].replace("_", " ")
    text = f"Title: {title}\nAbstract: {row['original_abstract']}"
    abstract_lookup[int(row['abstract_id'])] = text

print(f"Created lookup for {len(abstract_lookup)} abstracts.")

Creating abstract_id -> text lookup dictionary...
Created lookup for 363133 abstracts.


In [10]:
abstract_lookup[0]

'Title: How do lexical semantics affect translation? An empirical study\nAbstract: Neural machine translation (NMT) systems aim to map text from one language\ninto another. While there are a wide variety of applications of NMT, one of the\nmost important is translation of natural language. A distinguishing factor of\nnatural language is that words are typically ordered according to the rules of\nthe grammar of a given language. Although many advances have been made in\ndeveloping NMT systems for translating natural language, little research has\nbeen done on understanding how the word ordering of and lexical similarity\nbetween the source and target language affect translation performance. Here, we\ninvestigate these relationships on a variety of low-resource language pairs\nfrom the OpenSubtitles2016 database, where the source language is English, and\nfind that the more similar the target language is to English, the greater the\ntranslation performance. In addition, we study the impa

In [11]:
import random

def make_conversation(example):
    query_text = example['query_text']
    candidate_pool = example['candidate_pool']
    candidate_scores = example['candidate_scores']

    # Generate passages string as before
    passages_str_list = []
    for i, doc_id in enumerate(candidate_pool):
        doc_text = abstract_lookup.get(int(doc_id), "Error: Document text not found.")
        passages_str_list.append(f"[{i+1}] {doc_text}")
    passages_for_prompt = "\n\n---\n\n".join(passages_str_list)

    # Build the user prompt
    user_msg = (
        f"You are an expert academic paper reranker. "
        f"Your task is to re-order the given list of passages (from [1] to [{len(candidate_pool)}]) "
        f"based on their relevance to the query. Respond with only the ranking and nothing else.\n\n"
        f"Example output for 8 passages:\n"
        f"[2] > [5] > [4] > [8] > [6] > [1] > [3] > [7]\n\n"
        f"Query: {query_text}\n\n"
        f"Passages:\n{passages_for_prompt}\n\n"
        f"Your ranking (most to least relevant):"
    )

    # Build the assistant response (ground truth ranking)
    indexed_scores = [(i+1, score) for i, score in enumerate(candidate_scores)]
    sorted_indices = sorted(indexed_scores, key=lambda x: x[1], reverse=True)
    sorted_list_str = " > ".join(f"[{idx}]" for idx, _ in sorted_indices)

    conversation = [
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": sorted_list_str}
    ]

    return {"conversations": conversation}

def make_conversation_k(example):
    query_text = example['query_text']
    candidate_pool = example['candidate_pool']
    candidate_scores = example['candidate_scores']
    
    # 1. Identify Positives and Negatives
    positives = [(doc_id, score) for doc_id, score in zip(candidate_pool, candidate_scores) if score > 0]
    negatives = [(doc_id, score) for doc_id, score in zip(candidate_pool, candidate_scores) if score == 0]
    
    # 2. Configuration
    TARGET_SIZE = 20
    # Force the model to see mostly negatives, mimicking the LitSearch environment
    # LitSearch usually has 1 relevant doc per 20. 
    # Training with 4-5 positives helps the model learn comparison better than just 1.
    MAX_POSITIVES = 5 
    
    # 3. Select Documents
    random.shuffle(positives)
    random.shuffle(negatives)
    
    # Take up to MAX_POSITIVES
    selected_positives = positives[:MAX_POSITIVES]
    
    # Calculate how many negatives we need to fill the TARGET_SIZE
    num_negatives_needed = TARGET_SIZE - len(selected_positives)
    
    # Select negatives
    selected_negatives = negatives[:num_negatives_needed]
    
    # Combine
    selected_docs = selected_positives + selected_negatives
    
    # EDGE CASE: If we still don't have 20 docs (e.g., not enough negatives), 
    # fill back up with remaining positives if available
    if len(selected_docs) < TARGET_SIZE:
        remaining_positives = positives[MAX_POSITIVES:]
        needed = TARGET_SIZE - len(selected_docs)
        selected_docs += remaining_positives[:needed]

    # 4. Shuffle the final list so positives aren't always at the top in the input
    random.shuffle(selected_docs)
    
    # Extract for columns
    pool_k = [doc_id for doc_id, score in selected_docs]
    scores_k = [score for doc_id, score in selected_docs]
    
    # 5. Prepare the prompt
    passages_str_list = []
    for i, (doc_id, score) in enumerate(selected_docs):
        doc_text = abstract_lookup.get(int(doc_id), "Error: Document text not found.")
        doc_text = " ".join(doc_text.split()[:300]) 
        passages_str_list.append(f"[{i+1}] {doc_text}")

    passages_for_prompt = "\n\n---\n\n".join(passages_str_list)

    # Build User Prompt
    user_msg = (
        f"You are an expert academic paper reranker. "
        f"Your task is to re-order the given list of passages (from [1] to [{len(selected_docs)}]) "
        f"based on their relevance to the query. Respond with only the ranking and nothing else.\n\n"
        f"Example output for 8 passages:\n"
        f"[2] > [5] > [4] > [8] > [6] > [1] > [3] > [7]\n\n"
        f"Query: {query_text}\n\n"
        f"Passages:\n{passages_for_prompt}\n\n"
        f"Your ranking (most to least relevant):"
    )

    # Build Ground Truth
    # Sort by score (descending). 
    # Secondary sort by original index (ascending) ensures stable sorting for ties.
    indexed_scores = [(i+1, score) for i, score in enumerate(scores_k)]
    sorted_indices = sorted(indexed_scores, key=lambda x: (x[1], -x[0]), reverse=True)
    
    sorted_list_str = " > ".join(f"[{idx}]" for idx, _ in sorted_indices)

    conversation = [
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": sorted_list_str}
    ]

    return {
        "conversations": conversation,
        "pool_k": pool_k,
        "scores_k": scores_k
    }

def formatting_prompts_func_k(examples):
    texts = [] 
    new_pools = []
    new_scores = []
    
    num_examples = len(examples['query_text'])
    
    for i in range(num_examples):
        # 1. Reconstruct the single example
        single_example = {key: examples[key][i] for key in examples}
        
        # 2. Process
        processed = make_conversation_k(single_example)
        
        # 3. Apply Chat Template
        formatted_text = tokenizer.apply_chat_template(
            processed['conversations'],
            tokenize=False,
            add_generation_prompt=False
        )
        
        formatted_text = formatted_text.removeprefix('<bos>')
        
        # 4. Append to lists
        texts.append(formatted_text)
        new_pools.append(processed['pool_k'])
        new_scores.append(processed['scores_k'])
    
    # Return dictionary with all new columns
    return { 
        "text": texts, 
        "candidate_pool_k": new_pools, 
        "candidate_scores_k": new_scores 
    }

def formatting_prompts_func(examples):
    texts = [] # We need a list for the final formatted strings
    
    num_examples = len(examples['query_text'])
    
    for i in range(num_examples):
        # 1. Reconstruct the single example
        single_example = {key: examples[key][i] for key in examples}
        
        # 2. Build the structural conversation (List of Dicts)
        # This returns {'conversations': [{'role': 'user', ...}, ...]}
        processed = make_conversation_k(single_example)
        
        # 3. APPLY THE CHAT TEMPLATE (The missing step!)
        # This converts the List of Dicts into the specific string format for Gemma 3
        formatted_text = tokenizer.apply_chat_template(
            processed['conversations'],
            tokenize=False,
            add_generation_prompt=False
        )
        
        # 4. Remove the extra <bos> token if Unsloth adds its own
        formatted_text = formatted_text.removeprefix('<bos>')
        
        texts.append(formatted_text)
    
    # Return the field 'text', which is what the Trainer usually looks for
    return { "text": texts }

In [12]:
dataset = dataset.map(formatting_prompts_func_k, batched=True)

In [13]:
dataset[1]

{'query_text': 'Since the labeling process is time-consuming and very costly, before I actually create the dataset, I need to evaluate the expected quality of my data labeling process by conducting a feasibility study.. Overall, I need to conduct several feasibility studies to identify potential challenges and make a decision about whether or not to proceed.',
 'aspect_id': '762',
 'original_query_type': 'ML',
 'candidate_pool': [190473,
  43017,
  91148,
  160784,
  182295,
  180254,
  218656,
  73769,
  304686,
  161842,
  31284,
  43574,
  291900,
  29758,
  163902,
  193600,
  69186,
  91724,
  150101,
  104533,
  15958,
  183906,
  155760,
  127603,
  145529,
  237690,
  221824,
  336000,
  41603,
  81032,
  137361,
  220823,
  90780,
  180383,
  204960,
  135327,
  151719,
  17070,
  150703,
  11953,
  265906,
  201910,
  150199,
  23737,
  264889,
  182983,
  75471,
  41174,
  34019,
  14565,
  24295,
  155370,
  7915,
  160505,
  267001,
  154876,
  226054,
  319754,
  160526,


In [14]:
dataset[1]['text']


'<start_of_turn>user\nYou are an expert academic paper reranker. Your task is to re-order the given list of passages (from [1] to [20]) based on their relevance to the query. Respond with only the ranking and nothing else.\n\nExample output for 8 passages:\n[2] > [5] > [4] > [8] > [6] > [1] > [3] > [7]\n\nQuery: Since the labeling process is time-consuming and very costly, before I actually create the dataset, I need to evaluate the expected quality of my data labeling process by conducting a feasibility study.. Overall, I need to conduct several feasibility studies to identify potential challenges and make a decision about whether or not to proceed.\n\nPassages:\n[1] Title: Image recognition from raw labels collected without annotators Abstract: Image classification problems are typically addressed by first collecting examples with candidate labels, second cleaning the candidate labels manually, and third training a deep neural network on the clean examples. The manual labeling step i

In [15]:
from datasets import DatasetDict

print(f"Original dataset size: {len(dataset)}")

train_val_test_split = dataset.train_test_split(test_size=0.1, seed=42)

train_val_split = train_val_test_split['train'].train_test_split(test_size=(0.1/0.9), seed=42)

finetuning_splits = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test'],
    'test': train_val_test_split['test']
})

print(finetuning_splits)

Original dataset size: 1501
DatasetDict({
    train: Dataset({
        features: ['query_text', 'aspect_id', 'original_query_type', 'candidate_pool', 'candidate_scores', 'text', 'candidate_pool_k', 'candidate_scores_k'],
        num_rows: 1200
    })
    validation: Dataset({
        features: ['query_text', 'aspect_id', 'original_query_type', 'candidate_pool', 'candidate_scores', 'text', 'candidate_pool_k', 'candidate_scores_k'],
        num_rows: 150
    })
    test: Dataset({
        features: ['query_text', 'aspect_id', 'original_query_type', 'candidate_pool', 'candidate_scores', 'text', 'candidate_pool_k', 'candidate_scores_k'],
        num_rows: 151
    })
})


In [16]:
finetuning_splits['train'][100]

{'query_text': 'My plan is to develop a deep summarization model and an alignment model.',
 'aspect_id': '251',
 'original_query_type': 'Composite',
 'candidate_pool': [309761,
  158725,
  310279,
  61976,
  43574,
  127040,
  81985,
  136256,
  109123,
  162893,
  177235,
  165983,
  346228,
  160387,
  202383,
  216210,
  54432,
  273057,
  309926,
  216743,
  78505,
  9385,
  58539,
  2220,
  240823,
  91840,
  141507,
  203983,
  319696,
  149732,
  55016,
  97000,
  175850,
  360685,
  111864,
  128252,
  111871,
  161027,
  264967,
  47368,
  54027,
  82223,
  14641,
  279351,
  64314,
  131391,
  16191,
  216384,
  53577,
  328015,
  192342,
  61785,
  230234,
  45920,
  140642,
  299878,
  138097,
  89474,
  302471,
  203144,
  100234,
  127889,
  9109,
  270743,
  226204,
  311204,
  132519,
  174508,
  14774,
  127930,
  234436,
  94662,
  68562,
  60381,
  247262,
  46562,
  143331,
  125415,
  200687,
  102910,
  230456,
  21737,
  95832,
  109884,
  52583,
  213573,
  9736

In [17]:
print(f"Example of formatted train data:\n{finetuning_splits['train'][100]['text']}")


Example of formatted train data:
<start_of_turn>user
You are an expert academic paper reranker. Your task is to re-order the given list of passages (from [1] to [20]) based on their relevance to the query. Respond with only the ranking and nothing else.

Example output for 8 passages:
[2] > [5] > [4] > [8] > [6] > [1] > [3] > [7]

Query: My plan is to develop a deep summarization model and an alignment model.

Passages:
[1] Title: VideoSET: Video Summary Evaluation through Text Abstract: In this paper we present VideoSET, a method for Video Summary Evaluation through Text that can evaluate how well a video summary is able to retain the semantic information contained in its original video. We observe that semantics is most easily expressed in words, and develop a text-based approach for the evaluation. Given a video summary, a text representation of the video summary is first generated, and an NLP-based metric is then used to measure its semantic distance to ground-truth text summaries 

In [18]:
def calculate_length(example):
    tokens = tokenizer(text=example['text'], truncation=False)
    
    return {"token_length": len(tokens['input_ids'][0])}

print("Calculating token lengths for the training set...")
train_val_with_lengths = train_val_split.map(calculate_length, num_proc=4)

all_lengths = []
for split in train_val_with_lengths:
    all_lengths.extend(train_val_with_lengths[split]["token_length"])

max_tokens_in_set = max(all_lengths)
print(f"The maximum number of tokens is: {max_tokens_in_set}")

Calculating token lengths for the training set...
The maximum number of tokens is: 5963


In [ ]:
# import optuna
# import gc
# import torch
# from unsloth import FastModel

# def model_init(trial):
#     # 1. Clean up memory from previous runs (Crucial for single GPU!)
#     gc.collect()
#     torch.cuda.empty_cache()
    
#     # 2. Define the search space for LoRA parameters
#     # We define them here so we can pass them to get_peft_model
#     r_value = trial.suggest_categorical("peft_r", [8, 16, 32])
#     alpha_value = trial.suggest_categorical("peft_alpha", [16, 32, 64])
#     dropout_value = trial.suggest_float("peft_dropout", 0.0, 0.1)
    
#     # 3. Load the base model (Unsloth handles caching efficiently)
#     model, _ = FastModel.from_pretrained(
#         model_name = "unsloth/gemma-3-4b-it",
#         max_seq_length = 8192,
#         load_in_4bit = True,
#         load_in_8bit = False,
#     )
    
#     # 4. Apply LoRA with the trial's parameters
#     model = FastModel.get_peft_model(
#         model,
#         r = r_value,
#         lora_alpha = alpha_value,
#         lora_dropout = dropout_value,
#         target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
#         finetune_vision_layers = False,
#         finetune_language_layers = True,
#         finetune_attention_modules = True,
#         finetune_mlp_modules = True,
#         bias = "none",
#         random_state = 3407,
#     )
    
#     return model

In [ ]:
# def hp_space(trial):
#     return {
#         "learning_rate": trial.suggest_float("learning_rate", 1e-6, 2e-4, log=True),
#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
#         "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4]),
#         "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.1),
#     }

In [ ]:
# import optuna
# import gc
# import torch
# from unsloth import FastModel
# from unsloth.chat_templates import get_chat_template
# from trl import SFTTrainer, SFTConfig

# def objective(trial):
#     # 1. Clean up memory from previous trial
#     gc.collect()
#     torch.cuda.empty_cache()
    
#     # 2. Define Hyperparameters to Tune
#     # LoRA Params
#     r_value = trial.suggest_categorical("peft_r", [8, 16])
#     alpha_value = trial.suggest_categorical("peft_alpha", [16, 32]) # Keep alpha/r ratio reasonable (2x)
#     dropout_value = trial.suggest_float("peft_dropout", 0.0, 0.1)
    
#     # Training Params
#     lr_value = trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True)
#     batch_size_value = trial.suggest_categorical("batch_size", [8, 16])
    
#     # 3. Load Model (Must be done fresh every trial)
#     model, tokenizer = FastModel.from_pretrained(
#         model_name = "unsloth/gemma-3-4b-it",
#         max_seq_length = 8192,
#         load_in_4bit = True,
#         load_in_8bit = False,
#     )
    
#     # Apply Chat Template (Crucial!)
#     tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")
    
#     # Apply LoRA with trial parameters
#     model = FastModel.get_peft_model(
#         model,
#         r = r_value,
#         lora_alpha = alpha_value,
#         lora_dropout = dropout_value,
#         target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
#         finetune_vision_layers = False,
#         finetune_language_layers = True,
#         finetune_attention_modules = True,
#         finetune_mlp_modules = True,
#         bias = "none",
#         random_state = 3407,
#     )
    
#     # 4. Configure Trainer
#     # We use a smaller number of steps/epochs for tuning to save time
#     training_args = SFTConfig(
#         dataset_text_field="text",
#         max_seq_length=8192,
#         output_dir=f"hyperparam_results/trial_{trial.number}",
#         per_device_train_batch_size=batch_size_value,
#         gradient_accumulation_steps=1, 
#         num_train_epochs=1,      # 1 Epoch is enough to see convergence trends
#         learning_rate=lr_value,
#         logging_steps=10,
#         eval_strategy="steps",
#         eval_steps=20,           # Frequent eval to catch overfitting early
#         save_strategy="no",      # Don't save checkpoints to save disk space
#         optim="adamw_8bit",
#         weight_decay=0.01,
#         seed=3407,
#         report_to="none",
#     )
    
#     # Use the 'train_on_responses_only' wrapper manually if needed, 
#     # or rely on the fact that SFTTrainer handles the template if formatted correctly.
#     # Since we formatted the dataset with the template already, we pass it directly.
    
#     trainer = SFTTrainer(
#         model=model,
#         tokenizer=tokenizer,
#         train_dataset=finetuning_splits['train'],
#         eval_dataset=finetuning_splits['validation'],
#         args=training_args,
#     )
    
#     # Apply the response masking wrapper
#     from unsloth.chat_templates import train_on_responses_only
#     trainer = train_on_responses_only(
#         trainer,
#         instruction_part = "<start_of_turn>user\n",
#         response_part = "<start_of_turn>model\n",
#     )
    
#     # 5. Train
#     trainer.train()
    
#     # 6. Get Final Validation Loss
#     eval_stats = trainer.evaluate()
#     val_loss = eval_stats["eval_loss"]
    
#     # 7. Cleanup to prevent OOM
#     del model
#     del trainer
#     gc.collect()
#     torch.cuda.empty_cache()
    
#     return val_loss

# # Run the study
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=10) # Run 10 trials

# print("Best hyperparameters:", study.best_params)

[I 2025-12-05 21:04:57,578] A new study created in memory with name: no-name-64863ff5-8489-438e-8cf8-69845f72b3a5


==((====))==  Unsloth 2025.11.6: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA H200 NVL. Num GPUs = 1. Max memory: 139.801 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.07055564516573974.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 1 | Total steps = 75
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


...

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


Step,Training Loss,Validation Loss
20,0.265700,0.195823
40,0.145400,0.153320
60,0.135100,0.148629
80,0.139100,0.144463
100,0.140800,0.143418
120,0.145600,0.142862
140,0.136800,0.142227


[W 2025-12-06 03:08:14,887] Trial 9 failed with parameters: {'peft_r': 8, 'peft_alpha': 32, 'peft_dropout': 0.0603406222340807, 'learning_rate': 4.991177638697648e-05, 'batch_size': 8} because of the following error: The value nan is not acceptable.
[W 2025-12-06 03:08:14,890] Trial 9 failed with value nan.


Best hyperparameters: {'peft_r': 8, 'peft_alpha': 32, 'peft_dropout': 0.02370581206250282, 'learning_rate': 4.757632671241678e-05, 'batch_size': 16}


In [19]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        max_seq_length = 8192,
        dataset_text_field = "text",
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 20,
        warmup_ratio=0.06,
        # gradient_checkpointing = True,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 1e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
        bf16=True
    ),
)

In [20]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [21]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nYou are an expert academic paper reranker. Your task is to re-order the given list of passages (from [1] to [20]) based on their relevance to the query. Respond with only the ranking and nothing else.\n\nExample output for 8 passages:\n[2] > [5] > [4] > [8] > [6] > [1] > [3] > [7]\n\nQuery: The document should discuss the improvement of interpretability and explainability of machine learning models.\n\nPassages:\n[1] Title: Contrastive estimation reveals topic posterior information to linear models Abstract: Contrastive learning is an approach to representation learning that utilizes naturally occurring similar and dissimilar pairs of data points to find useful embeddings of data. In the context of document classification under topic modeling assumptions, we prove that contrastive learning is capable of recovering a representation of documents that reveals their underlying topic posterior information to linear models. We apply this procedure in a semi-supervi

In [22]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,501 | Num Epochs = 2 | Total steps = 188
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 262,307,840 of 4,562,387,312 (5.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.451400
2,1.495700
3,1.509700
4,1.356600
5,1.343500
6,1.026200
7,1.231800
8,0.924700
9,0.679400


In [24]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

model.save_pretrained_merged(f"models/{timestamp}/gemma-3-finetune", tokenizer)

Found HuggingFace hub cache directory: /home/cosmin/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 2 files from cache to `models/20251206_124536/gemma-3-finetune`: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:21<00:00, 10.87s/it]


Successfully copied all 2 files from cache to `models/20251206_124536/gemma-3-finetune`
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `models/20251206_124536/gemma-3-finetune`: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 53.39it/s]


Successfully copied all 1 files from cache to `models/20251206_124536/gemma-3-finetune`


Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.51s/it]


Unsloth: Merge process complete. Saved to `/DATA/j_andrei/Doris-Mae-Dataset/models/20251206_124536/gemma-3-finetune`


In [25]:
model.save_pretrained("models/{timestamp}/gemma-3-finetune")  # Local saving
tokenizer.save_pretrained("models/{timestamp}/gemma-3-finetune")

['models/{timestamp}/gemma-3-finetune/processor_config.json']